In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.optimizers import RMSprop
from keras.initializers import RandomNormal, Identity
from keras.datasets import mnist
from keras.utils import np_utils
import numpy as np
import random

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

Using TensorFlow backend.


# 1. 伪装曹雪芹

In [2]:
def load_text(path):
    with open(path, 'r') as f:
        text = f.read()
    text = text.decode('utf-8')
    
    return text

In [3]:
text = load_text('hongloumeng.txt')
print('corpus length:', len(text))
text = text[:600000]

corpus length: 861117


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 4312


In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 199987


In [6]:
range(0, 10, 3)

[0, 3, 6, 9]

In [22]:
def Generator(batch_size=128):
    while 1:
        for i in range(len(sentences)/batch_size-1):
            x = np.zeros((batch_size, maxlen, len(chars)), dtype=np.bool)
            y = np.zeros((batch_size, len(chars)), dtype=np.bool)
            batch_sentences = sentences[i*batch_size:(i+1)*batch_size]
            batch_label = next_chars[i*batch_size:(i+1)*batch_size]
            for j, sentence in enumerate(batch_sentences):
                for k, char in enumerate(sentence):
                    x[j, k, char_indices[char]] = 1
                y[j, char_indices[batch_label[j]]] = 1
            yield x, y

In [7]:
#print('Vectorization...')
#x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
#y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
#for i, sentence in enumerate(sentences):
    #for t, char in enumerate(sentence):
        #x[i, t, char_indices[char]] = 1
    #y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [9]:
#x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
#y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [10]:
#for i, sentence in enumerate(sentences):
    #for t, char in enumerate(sentence):
        #x[i, t, char_indices[char]] = 1
    #y[i, char_indices[next_chars[i]]] = 1

In [15]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [55]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [16]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [17]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype(np.float64)
    preds = np.log(preds) / diversity
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
def on_epoch_end(model, epoch):
    print()
    print('----- Generating text after Epoch: %d -----' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity: -----', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '" -----')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        print()
        print(generated)
        print()

In [ ]:
epochs = 60
generator = Generator(128)
for epoch in range(epochs):
    #model.fit(x, y, batch_size=128, epochs=1)
    model.fit_generator(generator, steps_per_epoch=len(sentences)/128, epochs=1)
    on_epoch_end(model, epoch)

Epoch 1/1
 232/1562 [===>..........................] - ETA: 92s - loss: 6.1664

# 2. 图像识别

In [28]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [29]:
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels, 1)
X_test = X_test.reshape(X_test.shape[0], num_pixels, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.

In [30]:
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

In [31]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

## 2.1 Simple RNN

In [34]:
model = Sequential()
model.add(SimpleRNN(100, input_shape=X_train.shape[1:], activation='tanh'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

In [34]:
model = Sequential()
model.add(SimpleRNN(100, input_shape=(num_pixels, 1), activation='tanh'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [37]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [38]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 170s - loss: 2.5153 - acc: 0.1006 - val_loss: 2.5110 - val_acc: 0.0980


## 2.2 Simple RNN with Relu and Identiy initialization
http://arxiv.org/pdf/1504.00941v2.pdf

In [37]:
model = Sequential()
model.add(SimpleRNN(100, kernel_initializer=RandomNormal(stddev=0.001), 
                    recurrent_initializer=Identity(gain=1.0), activation='relu',
                    input_shape=X_train.shape[1:]))
model.add(Dense(10))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

In [39]:
model = Sequential()
model.add(SimpleRNN(100, kernel_initializer=RandomNormal(stddev=0.01),
                   recurrent_initializer=Identity(gain=1.0),
                   activation='relu',
                   input_shape=(num_pixels, 1)))
model.add(Dense(10))
model.add(Activation('softmax'))

In [40]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [41]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 170s - loss: 0.0015 - acc: 0.0987 - val_loss: 1.1921e-07 - val_acc: 0.0980


## 2.3 LSTM

In [39]:
model = Sequential()
model.add(LSTM(100, input_shape=X_train.shape[1:]))
model.add(Dense(10))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

In [42]:
model = Sequential()
model.add(LSTM(100, input_shape=(num_pixels, 1)))
model.add(Dense(10))
model.add(Activation('softmax'))

In [43]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [40]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 1166s - loss: 2.3342 - acc: 0.1065 - val_loss: 2.3045 - val_acc: 0.1024


## 2.4 GRU

In [44]:
model = Sequential()
model.add(GRU(100, input_shape=X_train.shape[1:]))
model.add(Dense(10))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])

In [45]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 836s - loss: 2.3671 - acc: 0.1070 - val_loss: 2.4878 - val_acc: 0.1010
